In [35]:
import numpy as np
import pandas as pd
import glob

In [39]:
path_e = "DataForValidatingModels/Eugene"
path_c = "DataForValidatingModels/Cocoa"
path_g = "DataForValidatingModels/Golden"
eug_files = glob.glob(path_e + '/*.csv')
coc_files = glob.glob(path_c + '/*.csv')
gold_files = glob.glob(path_g + '/*.csv')
all_files = coc_files + eug_files + gold_files
#print(all_files)
#repeat for 3 and put them together
'''print(eug_files)
print(coc_files)
print(gold_files)'''

'print(eug_files)\nprint(coc_files)\nprint(gold_files)'

In [49]:
#loop over them and create dataframe for each with site number columns
for file in all_files:
    with open(file, "r") as f:
        f.readline() # skip line
        f.readline() # skip line
        lines = f.readlines()

    with open("cleaned/" + file, "w") as f:
        header = lines[0].split(",")
        f.write(",".join(header) + "\n") # write header
        for line in lines[1:]:
            f.write(",".join(line.split(",")[:len(header)]) + "\n") # write only labeled columns
li = []
for file in all_files:
    li.append("cleaned/" + file)
print(li)
#df = pd.concat(li)

KeyboardInterrupt: 

In [ ]:
def edit_time(elem):
    return elem.split('T')[0]

In [ ]:
df["Time Stamp"] = df["Time Stamp (local standard time) yyyy-mm-ddThh:mm:ss"].apply(edit_time)

In [ ]:
cols = ["Time Stamp",
        "POA irradiance CMP22 pyranometer (W/m2)",
        "Relative humidity (%RH)",
        "PV module soiling derate"]
df[cols].head()

In [ ]:
df_filtered = df.groupby(["Time Stamp"], as_index=False).first()
df_filtered["Site"] = "Eugene"
df_filtered["Site Index"] = 0
df_filtered[cols+["Site","Site Index"] ].head()